In [ ]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

In [ ]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
varApi = lumapi.MODE(hide = False)

##  Specification of the materials and dimensions of the guide

In [ ]:
# units
um = 1e-6
nm = 1e-9

In [ ]:
# Materials 
sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'

In [ ]:
# MMI x parameters
L = 57.42 * um
W = 7.0 * um

# Taper parameters
W1 = 450 * nm
W2 = 1.5 * um
L1 = 15 * um

# Waveguide parameters
wvg_width = 450 * nm
wvg_height = 220 * nm

# Waveguide length
wvg_length = 4.0 * um

# Bandwidth operation
cband = True
filename = "S_MMI_var_2x2_%d_um" % (L / um)

# C Band simulation: 1520 - 1575 nm
# L Band simulation: 1555 - 1635 nm


if cband:
    wvlngth_start = 1.520 * um
    wvlngth_stop = 1.575 * um
else:
    wvlngth_start = 1.555 * um
    wvlngth_stop = 1.635 * um

nmbrFrqPnts = 20


## Structure creation

In [ ]:
# Defining a Structure Group
varApi.addstructuregroup()
dev_layer = "GDS_LAYER 35:4"
varApi.set("name", dev_layer)

In [ ]:
varApi.switchtolayout()
varApi.deleteall()

# Add box
varApi.addrect()
varApi.addtogroup(dev_layer)
varApi.set("name", "box")
varApi.set("x", 0)
varApi.set("x span", L)
varApi.set("y", 0)
varApi.set("y span", W)
varApi.set("z", wvg_height / 2)
varApi.set("z span", wvg_height)
varApi.set("material", core_material)

# Tapers Configuration
for i in range(4):
    varApi.addobject("linear_taper")
    varApi.set("name", f"taper port {str(i + 1)}")
    varApi.set("thickness", wvg_height)
    varApi.set("angle_side", 90)
    varApi.set("width_r", W1)
    varApi.set("width_l", W2)
    varApi.set("len", L1)
    varApi.set("material", core_material)
    varApi.set("x", varApi.getnamed(f"{dev_layer}::box", "x max") + L1/2)
    varApi.set("y", 0)
    varApi.set("z", wvg_height / 2)

# Adjusting taper positions
varApi.select(f"{dev_layer}::taper port 1")
varApi.set("width_r", W2)
varApi.set("width_l", W1)
varApi.set("x", -varApi.getnamed(f"{dev_layer}::box", "x max") - L1/2)
varApi.set("y", W/6)

varApi.select(f"{dev_layer}::taper port 2")
varApi.set("width_r", W2)
varApi.set("width_l", W1)
varApi.set("x", -varApi.getnamed(f"{dev_layer}::box", "x max") - L1/2)
varApi.set("y", -W/6)

varApi.select(f"{dev_layer}::taper port 3")
varApi.set("y", W/6)

varApi.select(f"{dev_layer}::taper port 4")
varApi.set("y", -W/6)

# add waveguides
for i in range(4):
    varApi.addrect()
    varApi.addtogroup(dev_layer)
    varApi.set("name", f"wvguide {str(i+1)}")
    varApi.set("x", 0)
    varApi.set("x span", wvg_length)
    varApi.set("y", 0)
    varApi.set("y span", wvg_width)
    varApi.set("z", wvg_height / 2)
    varApi.set("z span", wvg_height)
    varApi.set("material", core_material)

# Adjusting the wvg positions
varApi.select(f"{dev_layer}::wvguide 1")
varApi.set("x", varApi.getnamed(f"{dev_layer}::box", "x min") - L1 - wvg_length/2)
varApi.set("y", W/6)

varApi.select(f"{dev_layer}::wvguide 2")
varApi.set("x", varApi.getnamed(f"{dev_layer}::box", "x min") - L1 - wvg_length/2)
varApi.set("y", -W/6)

varApi.select(f"{dev_layer}::wvguide 3")
varApi.set("x", varApi.getnamed(f"{dev_layer}::box", "x max") + L1 + wvg_length/2)
varApi.set("y", W/6)

varApi.select(f"{dev_layer}::wvguide 4")
varApi.set("x", varApi.getnamed(f"{dev_layer}::box", "x max") + L1 + wvg_length/2)
varApi.set("y", -W/6)

## Add solver